In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import numpy as np
import gc
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
import time
import pickle
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_poisson_deviance

import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [ ]:
# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()
    for store_id in STORES_IDS:
        #temp_df = pd.read_pickle('../input/m5-aux-models/test_'+store_id+'.pkl')
        temp_df = pd.read_pickle('../input/clg2aux/test_'+str(store_id)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
        
    return base_test

## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [ ]:

def make_lag_roll(LAG_DAY,grid):
    for lag in LAG_DAY:
        shift_day = lag[0]
        roll_wind = lag[1]
        gr = grid.groupby(["id"])["demand"]
        #lag_df = base_test[['id','d',TARGET]]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        grid[col_name] = gr.transform(lambda x: x.shift(shift_day).rolling(roll_wind,min_periods=1).mean())
    return grid

In [ ]:
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
#lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores

#LIMITS and const
TARGET      = 'demand'           # Our target (was sales)
START_TRAIN = 0             # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models


# AUX(pretrained) Models paths
#AUX_MODELS = '../input/m5-aux-models/'

# These features are the ones that lead to overfit and the ones that 
# are not in the test set 
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d','wday', 'year', 'month',TARGET]

#STORES ids
ORIGINAL = '../input/m5-forecasting-accuracy/'
#STORES_IDS =  pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
#STORES_IDS = list(STORES_IDS.unique())




#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])
        

In [ ]:
STORES_IDS = [0,1,2,3,4,5,6,7,8,9]

# DEMANDD FEATURES IN PREDICTION

In [ ]:
LAGS = [7, 28]
WINDOWS = [7, 14, 28]

def demand_features(df):
    """ Derive features from sales data and remove rows with missing values """
    
    for lag in LAGS:
        df[f'lag_t{lag}'] = df.groupby('id')['demand'].transform(lambda x: x.shift(lag)).astype("float32")
        for w in WINDOWS:
            df[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id')[f'lag_t{lag}'].transform(lambda x: x.rolling(w).mean()).astype("float32")
        
    return df

def demand_features_eval(df):
    """ Same as demand_features but for the step-by-step evaluation """
    out = df.groupby('id', sort=False).last()
    for lag in LAGS:
        out[f'lag_t{lag}'] = df.groupby('id', sort=False)['demand'].nth(-lag-1).astype("float32")
        for w in WINDOWS:
            out[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id', sort=False)['demand'].nth(list(range(-lag-w, -lag))).groupby('id', sort=False).mean().astype("float32")
    return out.reset_index()

In [ ]:
ORIGINAL = '../input/m5-forecasting-accuracy/'

submission = pd.read_csv(ORIGINAL+'sample_submission.csv')
#submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
#submission.to_csv('submission_v'+str(VER)+'.csv', index=False)

In [ ]:
FIRST = 1914
LENGTH = 28
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()
MODEL_FEATURES = [col for col in list(base_test) if col not in remove_features]

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day

#for PREDICT_DAY in range(1,29):   

for i, day in enumerate(np.arange(FIRST, FIRST + LENGTH)):
    print('Predict | Day:', day)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    #grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll,ROLS_SPLIT),df_parallelize_run(make_lag,ROLS_SPLIT)], axis=1)
    #
    
    grid_df = base_test.copy()
    
    test_day = demand_features_eval(grid_df[(grid_df.d <= day) & (grid_df.d >= day - max(LAGS) - max(WINDOWS))])
    for store_id in STORES_IDS:
    
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = '../input/clg2aux/lgb_model_'+str(store_id)+'_v'+str(VER)+'.bin' 
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==day
    
        
        base_test.loc[((base_test.d == day) & (base_test.store_id == store_id)),"demand"]= estimator.predict(test_day[test_day.store_id==store_id][MODEL_FEATURES])
       
                
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[base_test.d==day][['id',TARGET]]
    temp_df.columns = ['id','F'+str(i+1)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(i+1)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds




In [ ]:
all_preds.to_csv('predictions.csv', index=False)
all_preds = all_preds.assign(id=all_preds.id+"_validation")

In [ ]:
ORIGINAL = '../input/m5-forecasting-accuracy/'

submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(1)
submission.to_csv('submission_v1'+'.csv', index=False)